In [11]:
import os
import sys
import torch
import importlib
import numpy as np
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
from PIL import Image
from ipywidgets import widgets, interact
from IPython.display import display, clear_output
# Add the root directory to the path for imports
if os.path.dirname(os.getcwd()) not in sys.path:
    sys.path.append(os.path.dirname(os.getcwd()))
# ======== CONFIGURATION ========
# Set your paths and configuration here
CONFIG = {
    # Dataset path
    "dataset_path": "/home/stud/lavingal/storage/slurm/lavingal/LVSM/datasets/re10k/test/full_list.txt",
    
    # Specific checkpoint to load (set to empty to use latest from checkpoint_dir)
    "specific_checkpoint": "/home/stud/lavingal/storage/slurm/lavingal/experiments/checkpoints/LVSM_scene_decoder_only/ckpt_0000000000246000.pt",  # e.g. "/path/to/specific/ckpt_0000000000123456.pt"
    
    # Checkpoint directory (used if specific_checkpoint is empty)
     "checkpoint_dir": "../experiments/checkpoints/LVSM_scene_decoder_only",
    
    # Device to use
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    
    # Model configuration file (will be overridden with loaded values)
    "config_file": "/home/stud/lavingal/storage/slurm/lavingal/LVSM/configs/LVSM_scene_decoder_only.yaml",

    # Camera modification ranges
    "tx_range": (-6.0, 6.0),  # Translation X range
    "ty_range": (-6.0, 6.0),  # Translation Y range
    "tz_range": (-6.0, 6.0),  # Translation Z range
    "rx_range": (-90.0, 90.0),  # Rotation X range (degrees)
    "ry_range": (-90.0, 90.0),  # Rotation Y range (degrees)
    "rz_range": (-90.0, 90.0),  # Rotation Z range (degrees)
}
# ======== HELPER FUNCTIONS ========
def load_config(config_path=None):
    """Load configuration from YAML file"""
    if config_path and os.path.exists(config_path):
        import yaml
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
    else:
        # Use default config from the provided YAML content
        config = {
            "model": {
                "class_name": "model.LVSM_scene_decoder_only.Images2LatentScene",
                "image_tokenizer": {
                    "image_size": 256,
                    "patch_size": 8,
                    "in_channels": 9  # 3 RGB + 3 direction + 3 Reference
                },
                "target_pose_tokenizer": {
                    "image_size": 256,
                    "patch_size": 8,
                    "in_channels": 6  # 3 direction + 3 Reference
                },
                "transformer": {
                    "d": 768,
                    "d_head": 64,
                    "n_layer": 6,
                    "special_init": True,
                    "depth_init": True,
                    "use_qk_norm": True
                }
            },
            "training": {
                "amp_dtype": "bf16",
                "batch_size_per_gpu": 1,
                "center_crop": True,
                "scene_scale_factor": 1.35,
                "checkpoint_dir": CONFIG["checkpoint_dir"],
                "dataset_name": "data.dataset_scene.Dataset",
                "dataset_path": CONFIG["dataset_path"],
                "num_input_views": 2,
                "num_target_views": 6,
                "num_threads": 8,
                "num_views": 8,
                "num_workers": 1,
                "square_crop": True,
                "target_has_input": True,
                "use_amp": False,
                "use_tf32": False,
                "view_selector": {
                    "max_frame_dist": 192,
                    "min_frame_dist": 25
                }
            },
            "inference": {
                "render_video": False,
                "render_video_config": {
                    "traj_type": "interpolate",
                    "num_frames": 10,
                    "loop_video": False,
                    "order_poses": False
                }
            }
        }
    
    # Convert to EasyDict for easier access
    config = edict(config)
    
    # Override with our specific settings
    config.training.dataset_path = CONFIG["dataset_path"]
    config.training.checkpoint_dir = CONFIG["checkpoint_dir"]
    config.training.batch_size_per_gpu = 1
    config.training.num_workers = 1
    config.training.use_amp = False
    
    return config
def load_model(config):
    """Load the LVSM model and checkpoint"""
    # Set up mock distributed environment to avoid DDP-related errors
    import torch.distributed as dist
    if not dist.is_available() or not dist.is_initialized():
        import random
        # Use random port to avoid conflicts
        random_port = random.randint(29500, 65000)
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = str(random_port)
        os.environ['RANK'] = '0'
        os.environ['WORLD_SIZE'] = '1'
        if dist.is_available():
            try:
                dist.init_process_group(backend='gloo', rank=0, world_size=1)
            except RuntimeError:
                print("Warning: Could not initialize distributed environment. Continuing without it.")
    
    # Initialize LPIPS separately to avoid distributed initialization errors
    import lpips
    # Suppress future warnings
    import warnings
    warnings.filterwarnings('ignore', category=FutureWarning)
    
    # Create a custom class that inherits from the original but overrides problematic methods
    module, class_name = config.model.class_name.rsplit(".", 1)
    LVSM_Base = importlib.import_module(module).__dict__[class_name]
    
    class LVSMSingleDevice(LVSM_Base):
        def __init__(self, config):
            # Save original config settings
            orig_l2_weight = config.training.l2_loss_weight
            orig_lpips_weight = config.training.lpips_loss_weight
            orig_perceptual_weight = config.training.perceptual_loss_weight
            
            # Temporarily disable loss components to avoid DDP initialization
            config.training.l2_loss_weight = 1.0
            config.training.lpips_loss_weight = 0.0
            config.training.perceptual_loss_weight = 0.0
            
            # Initialize the base class
            super().__init__(config)
            
            # Restore original config settings
            config.training.l2_loss_weight = orig_l2_weight
            config.training.lpips_loss_weight = orig_lpips_weight
            config.training.perceptual_loss_weight = orig_perceptual_weight
    
    # Create model instance
    model = LVSMSingleDevice(config).to(CONFIG["device"])
    
    # Load checkpoint
    ckpt_path = CONFIG["specific_checkpoint"] if CONFIG["specific_checkpoint"] else config.training.checkpoint_dir
    model.load_ckpt(ckpt_path)
    model.eval()
    
    return model
def load_dataset(config):
    """Load the dataset"""
    dataset_name = config.training.dataset_name
    module, class_name = dataset_name.rsplit(".", 1)
    Dataset = importlib.import_module(module).__dict__[class_name]
    dataset = Dataset(config)
    return dataset
def show_image_grid(images, titles=None, figsize=(15, 15), rows=None, cols=None):
    """Display a grid of images"""
    if isinstance(images, torch.Tensor):
        # Convert from bfloat16 to float32 if needed
        if images.dtype == torch.bfloat16:
            images = images.to(torch.float32)
        # Convert from tensor [B, C, H, W] to numpy [B, H, W, C]
        images = images.detach().cpu().numpy().transpose(0, 2, 3, 1)
    
    # Determine grid size
    n_images = len(images)
    if rows is None and cols is None:
        cols = int(np.ceil(np.sqrt(n_images)))
        rows = int(np.ceil(n_images / cols))
    elif rows is None:
        rows = int(np.ceil(n_images / cols))
    elif cols is None:
        cols = int(np.ceil(n_images / rows))
    
    # Create figure
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if rows * cols == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    # Plot images
    for i, ax in enumerate(axes):
        if i < n_images:
            if images[i].shape[-1] == 1:  # Grayscale
                ax.imshow(images[i].squeeze(), cmap='gray')
            else:
                # Ensure values are in proper range for display
                img = np.clip(images[i], 0, 1)
                ax.imshow(img)
            
            if titles is not None and i < len(titles):
                ax.set_title(titles[i])
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
def process_c2w_matrix(c2w):
    """Process camera-to-world matrix for display"""
    # Extract rotation and translation
    rotation = c2w[:3, :3]
    translation = c2w[:3, 3]
    
    # Convert rotation to Euler angles (in degrees)
    from scipy.spatial.transform import Rotation
    euler = Rotation.from_matrix(rotation.cpu().numpy()).as_euler('xyz', degrees=True)
    
    return {
        'translation': translation.cpu().numpy(),
        'rotation_euler_deg': euler,
        'full_matrix': c2w.cpu().numpy()
    }
def get_ray_bundle(fxfycxcy, c2w, height, width, device="cpu"):
    """Generate rays for a camera view"""
    # Create pixel coordinates
    i, j = torch.meshgrid(
        torch.arange(width, device=device),
        torch.arange(height, device=device),
        indexing="ij"
    )
    i = i.t()
    j = j.t()
    
    # Convert to normalized device coordinates
    fx, fy, cx, cy = fxfycxcy
    dirs = torch.stack(
        [
            (i - cx) / fx,
            -(j - cy) / fy,
            -torch.ones_like(i),
        ],
        dim=-1,
    )
    
    # Transform ray directions to world space
    rays_d = dirs @ c2w[:3, :3].t()
    rays_d = rays_d / torch.linalg.norm(rays_d, dim=-1, keepdim=True)
    
    # Get ray origins from camera position
    rays_o = c2w[:3, 3].expand(rays_d.shape)
    
    return rays_o, rays_d

In [12]:
def create_camera_in_world(position, rotation_euler_deg, device=None):
    """
    Create a camera-to-world matrix directly from position and rotation in world coordinates
    
    Args:
        position: [x, y, z] camera position in world coordinates
        rotation_euler_deg: [rx, ry, rz] rotation in degrees (Euler angles, XYZ order)
        device: Device to place tensor on (default: CONFIG["device"])
        
    Returns:
        c2w: 4x4 camera-to-world transformation matrix
    """
    from scipy.spatial.transform import Rotation
    if device is None:
        device = CONFIG["device"]
    
    # Create rotation matrix from Euler angles
    r = Rotation.from_euler('xyz', rotation_euler_deg, degrees=True)
    rotation_matrix = r.as_matrix()
    
    # Create camera-to-world matrix
    c2w = torch.eye(4)
    c2w[:3, :3] = torch.tensor(rotation_matrix, dtype=torch.float32)
    c2w[:3, 3] = torch.tensor(position, dtype=torch.float32)
    
    return c2w.to(device)

In [13]:
# ======== MAIN CODE ========
# Load configuration
config = load_config(CONFIG.get("config_file"))
# Modify config to avoid distributed training issues
config.training.use_amp = False
config.training.use_tf32 = False
# Load dataset first
dataset = load_dataset(config)
print(f"Dataset loaded with {len(dataset)} scenes")
# Now load model with modified configuration
try:
    model = load_model(config)
    print(f"Model loaded from {CONFIG['specific_checkpoint'] if CONFIG['specific_checkpoint'] else config.training.checkpoint_dir}")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    import traceback
    traceback.print_exc()
# ======== SCENE SELECTION ========
def select_scene(scene_idx=0):
    """Select a scene and display input and target views"""
    if scene_idx < 0 or scene_idx >= len(dataset):
        print(f"Scene index out of range. Please choose between 0 and {len(dataset)-1}")
        return None
    
    try:
        # Get scene data
        scene_data = dataset[scene_idx]
        
        # Move to device
        scene_data = {k: v.to(CONFIG["device"]) if isinstance(v, torch.Tensor) else v 
                    for k, v in scene_data.items()}
        
        # Get batch dimension right
        for k, v in scene_data.items():
            if isinstance(v, torch.Tensor) and v.dim() > 0:
                scene_data[k] = v.unsqueeze(0)
        
        # Process data through model's process_data function
        with torch.no_grad():
            input_data, target_data = model.process_data(
                scene_data, 
                has_target_image=True, 
                target_has_input=config.training.target_has_input, 
                compute_rays=True
            )
    except Exception as e:
        print(f"Error processing scene {scene_idx}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
    
    # Display input views
    print("\n=== Input Views ===")
    input_images = input_data.image.squeeze(0)  # [v, c, h, w]
    show_image_grid(input_images, titles=[f"Input View {i}" for i in range(input_images.shape[0])])
    
    # Display camera parameters for input views
    print("\n=== Input Camera Parameters ===")
    for i in range(input_data.c2w.shape[1]):
        c2w_info = process_c2w_matrix(input_data.c2w[0, i])
        print(f"Input View {i}:")
        print(f"  Translation: {c2w_info['translation']}")
        print(f"  Rotation (Euler XYZ, degrees): {c2w_info['rotation_euler_deg']}")
        print()
    
    # Display target views
    print("\n=== Target Views ===")
    target_images = target_data.image.squeeze(0)  # [v, c, h, w]
    show_image_grid(target_images, titles=[f"Target View {i}" for i in range(target_images.shape[0])])
    
    # Display camera parameters for target views
    print("\n=== Target Camera Parameters ===")
    for i in range(target_data.c2w.shape[1]):
        c2w_info = process_c2w_matrix(target_data.c2w[0, i])
        print(f"Target View {i}:")
        print(f"  Translation: {c2w_info['translation']}")
        print(f"  Rotation (Euler XYZ, degrees): {c2w_info['rotation_euler_deg']}")
        print()
    
    return {"input": input_data, "target": target_data, "scene_data": scene_data}

Dataset loaded with 7286 scenes
Model loaded from /home/stud/lavingal/storage/slurm/lavingal/experiments/checkpoints/LVSM_scene_decoder_only/ckpt_0000000000246000.pt


In [14]:
# ======== CUSTOM CAMERA PARAMETERS (RELATIVE METHOD) ========
def create_custom_camera(base_camera, tx=0.0, ty=0.0, tz=0.0, rx=0.0, ry=0.0, rz=0.0):
    """Create a custom camera by modifying the base camera parameters"""
    # Start with the base camera
    c2w = base_camera.clone()
    
    # Create rotation matrices for the adjustments
    from scipy.spatial.transform import Rotation
    r = Rotation.from_euler('xyz', [rx, ry, rz], degrees=True)
    rot_matrix = torch.tensor(r.as_matrix(), dtype=c2w.dtype, device=c2w.device)
    
    # Apply rotation to the existing rotation matrix
    c2w[:3, :3] = torch.matmul(rot_matrix, c2w[:3, :3])
    
    # Apply translation adjustments
    c2w[0, 3] += tx
    c2w[1, 3] += ty
    c2w[2, 3] += tz
    
    return c2w

In [15]:
def render_with_custom_camera(scene_data, input_data, target_data, base_camera_idx=0, tx=0.0, ty=0.0, tz=0.0, rx=0.0, ry=0.0, rz=0.0):
    """Render the scene with a custom camera (relative to base camera)"""
    try:
        with torch.no_grad():
            # Create custom camera
            base_c2w = input_data.c2w[0, base_camera_idx].clone()
            custom_c2w = create_custom_camera(base_c2w, tx, ty, tz, rx, ry, rz)
            
            # Create a copy of the original scene data
            raw_scene_data = {k: v.clone() if isinstance(v, torch.Tensor) else v 
                             for k, v in scene_data["scene_data"].items()}
            
            # Modify ALL target views to use our custom camera
            num_input_views = config.training.num_input_views
            num_target_views = config.training.num_target_views
            
            for i in range(num_target_views):
                target_view_idx = num_input_views + i
                raw_scene_data["c2w"][0, target_view_idx] = custom_c2w
            
            # Use autocast like in your inference.py
            with torch.cuda.amp.autocast(
                enabled=True,
                dtype=torch.bfloat16
            ):
                # Turn off gradient checkpointing by modifying config temporarily
                orig_checkpoint_every = config.training.grad_checkpoint_every
                config.training.grad_checkpoint_every = 999999
                
                # Process through model
                result = model(raw_scene_data, has_target_image=False)
                
                # Restore config
                config.training.grad_checkpoint_every = orig_checkpoint_every
            
            # Print shape info for debugging
            print(f"Result.render shape: {result.render.shape}")
            
            # Get rendered image
            rendered_image = result.render[:, 0].to(torch.float32)
            
            # Display camera parameters
            c2w_info = process_c2w_matrix(custom_c2w)
            print("\n=== Custom Camera Parameters ===")
            print(f"  Translation: {c2w_info['translation']}")
            print(f"  Rotation (Euler XYZ, degrees): {c2w_info['rotation_euler_deg']}")
            print(f"  Relative to input view: {base_camera_idx}")
            
            # Display the rendered image
            print("\n=== Custom Camera Rendering ===")
            show_image_grid(rendered_image, titles=["Rendered View"])
            
            return rendered_image
            
    except Exception as e:
        print(f"Error rendering with custom camera: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [16]:
# ======== WORLD COORDINATE CAMERA CONTROL ========
def render_with_world_camera(scene_data, position, rotation_euler_deg):
    """
    Render the scene with a camera defined directly in world coordinates
    
    Args:
        scene_data: The scene data dictionary
        position: [x, y, z] camera position in world coordinates
        rotation_euler_deg: [rx, ry, rz] rotation in degrees (Euler angles, XYZ order)
        
    Returns:
        rendered_image: The rendered image tensor
    """
    try:
        with torch.no_grad():
            # Create camera directly in world coordinates
            custom_c2w = create_camera_in_world(position, rotation_euler_deg)
            
            # Create a copy of the original scene data
            raw_scene_data = {k: v.clone() if isinstance(v, torch.Tensor) else v 
                            for k, v in scene_data["scene_data"].items()}
            
            # We'll modify all target views to our new camera
            # This ensures the model isn't using other target views with different cameras
            num_input_views = config.training.num_input_views
            num_target_views = config.training.num_target_views
            
            # Set all target views to use our custom camera
            for i in range(num_target_views):
                target_view_idx = num_input_views + i
                raw_scene_data["c2w"][0, target_view_idx] = custom_c2w
            
            # Use autocast like in your inference.py
            with torch.cuda.amp.autocast(
                enabled=True,
                dtype=torch.bfloat16
            ):
                # Turn off gradient checkpointing by modifying config temporarily
                orig_checkpoint_every = config.training.grad_checkpoint_every
                config.training.grad_checkpoint_every = 999999
                
                # Process through model
                result = model(raw_scene_data, has_target_image=False)
                
                # Restore config
                config.training.grad_checkpoint_every = orig_checkpoint_every
            
            # Print shape info for debugging
            print(f"Result.render shape: {result.render.shape}")
            
            # Get rendered image - just the first target view
            rendered_image = result.render[:, 0].to(torch.float32)
            
            # Display camera parameters
            c2w_info = process_c2w_matrix(custom_c2w)
            print("\n=== World Camera Parameters ===")
            print(f"  Position: {position}")
            print(f"  Rotation (Euler XYZ, degrees): {rotation_euler_deg}")
            
            # Display the rendered image
            print("\n=== World Camera Rendering ===")
            show_image_grid(rendered_image, titles=["Rendered View"])
            
            return rendered_image
            
    except Exception as e:
        print(f"Error rendering with world camera: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [17]:
# ======== INTERACTIVE WIDGET ========
def run_interactive_demo():
    """Run an interactive demo to select scenes and customize camera parameters"""
    scene_output = None
    
    def on_scene_select(scene_idx):
        nonlocal scene_output
        clear_output(wait=True)
        scene_output = select_scene(scene_idx)
    
    # Create scene selection widget
    scene_select = widgets.IntSlider(
        value=0,
        min=0,
        max=len(dataset)-1,
        step=1,
        description='Scene Index:',
        continuous_update=False
    )
    
    # Connect the callback
    scene_select.observe(lambda change: on_scene_select(change['new']), names='value')
    
    # Display the widget
    display(scene_select)
    
    # Initialize with the first scene
    on_scene_select(0)

In [18]:
# ======== CUSTOM PARAMS CELLS ========
def render_with_custom_params(scene_idx, base_camera_idx=0, tx=0.0, ty=0.0, tz=0.0, rx=0.0, ry=0.0, rz=0.0):
    """Render with hardcoded custom camera parameters (relative method)"""
    # Get scene data
    scene_output = select_scene(scene_idx)
    if scene_output:
        # Render with custom parameters
        render_with_custom_camera(
            scene_output,
            scene_output["input"],
            scene_output["target"],
            base_camera_idx=base_camera_idx,
            tx=tx, ty=ty, tz=tz,
            rx=rx, ry=ry, rz=rz
        )

In [19]:
def render_with_world_params(scene_idx, position, rotation_euler_deg):
    """Render with hardcoded world coordinate camera parameters"""
    # Get scene data
    scene_output = select_scene(scene_idx)
    if scene_output:
        # Render with world coordinate parameters
        render_with_world_camera(
            scene_output,
            position=position,
            rotation_euler_deg=rotation_euler_deg
        )

In [20]:
# ======== CAMERA PARAMETER SWEEP (RELATIVE METHOD) ========
def run_parameter_sweep_relative(scene_idx=5, base_camera_idx=0, parameter="TY", 
                                start_value=-0.2, end_value=0.2, increment=0.002, fps=6,
                                output_dir="sweep_vids"):
    """
    Run a camera parameter sweep using the relative method
    
    Args:
        scene_idx: Index of the scene to render
        base_camera_idx: Which input camera to use as base
        parameter: Parameter to sweep ("TX", "TY", "TZ", "RX", "RY", "RZ")
        start_value: Starting value for the parameter
        end_value: Ending value for the parameter
        increment: Increment size for the parameter
        fps: Frames per second for the output video
        output_dir: Directory to save the output videos
    """
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    import imageio
    import time

    # Make sure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Time the process
    start_time = time.time()

    # Get scene data (only once)
    scene_output = select_scene(scene_idx)

    if scene_output:
        # Prepare for image collection
        frames = []
        parameter_values = []
        current_value = start_value
        
        # Show progress
        total_frames = int((end_value - start_value) / increment) + 1
        print(f"Starting parameter sweep of {parameter} from {start_value} to {end_value} with increment {increment}")
        print(f"Rendering approximately {total_frames} frames...")
        
        # Parameter sweep loop
        while current_value <= end_value:
            # Set the camera parameters according to which parameter we're sweeping
            tx, ty, tz, rx, ry, rz = 1.0, 0.0, 0.0, 0.0, 0.0, 0.0
            
            if parameter == "TX":
                tx = current_value
            elif parameter == "TY":
                ty = current_value
            elif parameter == "TZ":
                tz = current_value
            elif parameter == "RX":
                rx = current_value
            elif parameter == "RY": 
                ry = current_value
            elif parameter == "RZ":
                rz = current_value
            
            # Render with the current parameter value
            print(f"Rendering {parameter}={current_value:.2f}...")
            rendered_img = render_with_custom_camera(
                scene_output,
                scene_output["input"],
                scene_output["target"],
                base_camera_idx=base_camera_idx,
                tx=tx, ty=ty, tz=tz,
                rx=rx, ry=ry, rz=rz
            )
            
            # Convert tensor to numpy array and append to frames
            if rendered_img is not None:
                # Check the shape of the tensor to handle it correctly
                print(f"Rendered image shape: {rendered_img.shape}")
                
                # The render_with_custom_camera function likely returns a tensor with shape [1, 3, H, W]
                # We need to squeeze out any batch dimension if it exists
                if len(rendered_img.shape) == 4:
                    rendered_img = rendered_img.squeeze(0)  # Remove batch dimension if present
                
                # Now convert from tensor [C,H,W] to numpy [H,W,C]
                img_np = rendered_img.permute(1, 2, 0).cpu().numpy()
                # Clip values to valid range
                img_np = np.clip(img_np, 0, 1)
                frames.append((img_np * 255).astype(np.uint8))
                parameter_values.append(current_value)
            
            # Increment the parameter
            current_value += increment
        
        # Create the video file
        if frames:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            video_filename = os.path.join(output_dir, 
                                        f"{parameter}_sweep_{start_value}_to_{end_value}_scene{scene_idx}.mp4")
            
            print(f"Creating video with {len(frames)} frames...")
            imageio.mimsave(video_filename, frames, fps=fps)
            
            # Also create a GIF for easy viewing
            gif_filename = os.path.join(output_dir, 
                                    f"{parameter}_sweep_{start_value}_to_{end_value}_scene{scene_idx}.gif")
            imageio.mimsave(gif_filename, frames, fps=fps, loop=0)
            
            print(f"Video saved to: {video_filename}")
            print(f"GIF saved to: {gif_filename}")
            
            # Create a parameter value vs. frame number plot to help understand the relationship
            plt.figure(figsize=(10, 5))
            plt.plot(range(len(parameter_values)), parameter_values)
            plt.xlabel("Frame Number")
            plt.ylabel(f"{parameter} Value")
            plt.title(f"{parameter} Sweep Progression")
            plt.grid(True)
            plot_filename = os.path.join(output_dir, 
                                        f"{parameter}_sweep_plot_scene{scene_idx}.png")
            plt.savefig(plot_filename)
            plt.close()
            
            elapsed_time = time.time() - start_time
            print(f"Parameter sweep completed in {elapsed_time:.2f} seconds.")
            print(f"Generated {len(frames)} frames.")
        else:
             print("No frames were generated. Check for errors during rendering.")
    else:
        print(f"Failed to load scene {SCENE_INDEX}")

In [ ]:
run_parameter_sweep_relative(scene_idx=5, base_camera_idx=0, parameter="TX", 
                                start_value=-0.2, end_value=0.2, increment=0.002, fps=6,
                                output_dir="v2_sweep_vids")

SyntaxError: invalid syntax (55553674.py, line 3)